In [2]:
import os
import pandas as pd
import sqlite3

class CricketDatabaseManager:
    def __init__(self, db_path, datasets_dir):
        self.db_path = db_path
        self.datasets_dir = datasets_dir
        self.conn = None

    def connect(self):
        """Establish a connection to the SQLite database."""
        try:
            self.conn = sqlite3.connect(self.db_path)
            print("Connected to SQLite database successfully.")
        except sqlite3.Error as e:
            print(f"Error connecting to database: {e}")

    def close_connection(self):
        """Close the SQLite database connection."""
        if self.conn:
            self.conn.close()
            print("Database connection closed.")

    def create_tables(self):
        """Create tables for Test, ODI, and T20 matches."""
        if not self.conn:
            print("No database connection.")
            return
        
        cursor = self.conn.cursor()
        tables = {
            "test_matches": """
            CREATE TABLE IF NOT EXISTS test_matches (
                match_id INTEGER PRIMARY KEY AUTOINCREMENT,
                innings TEXT,
                revision TEXT,
                balls_per_over INTEGER,
                city TEXT,
                dates TEXT,
                event_match_number TEXT,
                event_name TEXT,
                gender TEXT,
                match_type TEXT,
                match_type_number INTEGER,
                officials_match_referees TEXT,
                officials_reserve_umpires TEXT,
                officials_tv_umpires TEXT,
                officials_umpires TEXT,
                outcome_by_runs INTEGER,
                outcome_winner TEXT,
                player_of_match TEXT,
                season TEXT,
                team_type TEXT
            );
            """,
            "odi_matches": """
            CREATE TABLE IF NOT EXISTS odi_matches (
                match_id INTEGER PRIMARY KEY AUTOINCREMENT,
                innings TEXT,
                data_version TEXT,
                created TEXT,
                revision TEXT,
                balls_per_over INTEGER,
                city TEXT,
                dates TEXT,
                event_match_number TEXT,
                event_name TEXT,
                gender TEXT,
                match_type TEXT,
                match_type_number INTEGER,
                officials_match_referees TEXT,
                officials_reserve_umpires TEXT,
                officials_tv_umpires TEXT,
                officials_umpires TEXT,
                outcome_by_runs INTEGER,
                outcome_winner TEXT,
                overs REAL,
                player_of_match TEXT,
                season TEXT
            );
            """,
            "t20_matches": """
            CREATE TABLE IF NOT EXISTS t20_matches (
                match_id INTEGER PRIMARY KEY AUTOINCREMENT,
                innings TEXT,
                data_version TEXT,
                created TEXT,
                revision TEXT,
                balls_per_over INTEGER,
                dates TEXT,
                event_match_number TEXT,
                event_name TEXT,
                gender TEXT,
                match_type TEXT,
                match_type_number INTEGER,
                officials_match_referees TEXT,
                officials_reserve_umpires TEXT,
                officials_tv_umpires TEXT,
                officials_umpires TEXT,
                outcome_by_wickets INTEGER,
                outcome_winner TEXT,
                overs REAL,
                player_of_match TEXT,
                season TEXT,
                team_type TEXT
            );
            """
        }

        try:
            for table_name, create_table_sql in tables.items():
                cursor.execute(create_table_sql)
            self.conn.commit()
            print("Tables created successfully.")
        except sqlite3.Error as e:
            print(f"Error creating tables: {e}")

    def insert_data(self, match_type, csv_path, row_limit=500):
        """Insert data into the appropriate table based on match type."""
        if not self.conn:
            print("No database connection.")
            return

        columns_map = {
            "test": ['innings', 'revision', 'balls_per_over', 'city', 'dates', 'event_match_number', 'event_name', 'gender', 'match_type',
                     'match_type_number', 'outcome_by_runs', 'outcome_winner',
                     'player_of_match', 'season', 'team_type'],
            "odi": ['innings', 'revision', 'balls_per_over',
                    'city', 'dates', 'event_match_number', 'event_name', 'gender', 'match_type',
                    'match_type_number', 'outcome_by_runs', 'outcome_winner',
                    'overs', 'player_of_match', 'season'],
            "t20": ['innings', 'revision', 'balls_per_over',
                    'dates', 'event_match_number', 'event_name', 'gender', 'match_type',
                    'match_type_number', 'outcome_by_wickets', 'outcome_winner',
                    'overs', 'player_of_match', 'season', 'team_type']
        
        }

        try:
            print(f"Reading data from {csv_path} for {match_type} matches.")
            df = pd.read_csv(csv_path, low_memory=False)
            print(f"Data read successfully. Total rows: {len(df)}")
            valid_columns = columns_map.get(match_type, [])
            df = df[valid_columns].iloc[:row_limit]
            print(f"Inserting {len(df)} rows into {match_type}_matches table.")
            df.to_sql(f"{match_type}_matches", self.conn, if_exists='append', index=False)
            print(f"Inserted {len(df)} rows into {match_type}_matches table successfully.")
        except Exception as e:
            print(f"Error inserting data for {match_type}: {e}")

def main():
    db_manager = CricketDatabaseManager("cricket_data.sqlite", "C:\\Users\\user\\Desktop\\DS_Cricsheet_Match_Analysis\\datasets")
    db_manager.connect()

    try:
        # Create tables
        db_manager.create_tables()

        # Insert data for all match types with the correct path
        match_types = {
            "test": os.path.join("C:\\Users\\user\\Desktop\\DS_Cricsheet_Match_Analysis\\datasets", "tests_matches.csv"),
            "odi": os.path.join("C:\\Users\\user\\Desktop\\DS_Cricsheet_Match_Analysis\\datasets", "odis_matches.csv"),
            "t20": os.path.join("C:\\Users\\user\\Desktop\\DS_Cricsheet_Match_Analysis\\datasets", "t20s_matches.csv")
        }

        for match_type, csv_file_path in match_types.items():
            if os.path.exists(csv_file_path):
                print(f"Inserting data for {match_type} matches from {csv_file_path}")
                db_manager.insert_data(match_type, csv_file_path, row_limit=500)
            else:
                print(f"File not found: {csv_file_path}")
    finally:
        db_manager.close_connection()

if __name__ == "__main__":
    main()


Connected to SQLite database successfully.
Tables created successfully.
Inserting data for test matches from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\tests_matches.csv
Reading data from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\tests_matches.csv for test matches.
Data read successfully. Total rows: 845
Inserting 500 rows into test_matches table.
Inserted 500 rows into test_matches table successfully.
Inserting data for odi matches from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\odis_matches.csv
Reading data from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\odis_matches.csv for odi matches.
Data read successfully. Total rows: 2860
Inserting 500 rows into odi_matches table.
Inserted 500 rows into odi_matches table successfully.
Inserting data for t20 matches from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\t20s_matches.csv
Reading data from C:\Users\user\Desktop\DS_Cricsheet_Match_Analysis\datasets\t20s_match